In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore


from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser


import os
from dotenv import dotenv_values


In [21]:
config = dotenv_values(dotenv_path='../.env')

In [22]:
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

# Definição de LLMs

In [23]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4, max_tokens=500)

# Carregar PDF e instanciar

In [24]:
path_file = './os-sertoes.pdf'

loader = PyPDFLoader(path_file, extract_images=False)

pages = loader.load_and_split()

# Definição de splitter

In [26]:
chilld_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [27]:
store = InMemoryStore()

vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="parent_rag_db"
)

In [28]:
parent_document_retriever = ParentDocumentRetriever(
  child_splitter=chilld_splitter,
  parent_splitter=text_splitter,
  vectorstore=vectorstore,
  docstore=store
)

In [29]:
parent_document_retriever.add_documents(pages, ids=None)

In [30]:
template_quests = '''
Você é um especialista no livro Os sertões de Euclides da Cunha. Deve responder objetivamente a questão do usuario com base no conteudo fornecido de contexto

Questão do usuario: {query}


Conteudo: {content}
'''

rag_prompt = ChatPromptTemplate.from_template(template_quests)

In [31]:
setup_retrievel = RunnableParallel({
    "query": RunnablePassthrough(),
    "content": parent_document_retriever,
})

In [32]:
output_parser = StrOutputParser()

In [33]:
parent_chain_retriever = setup_retrievel | rag_prompt | llm | output_parser

# Perguntas

In [34]:
parent_chain_retriever.invoke('Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?')

'A visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino é de um local marcado por condições extremas e adversas, onde os elementos físicos como o clima, a geologia e a topografia se influenciam mutuamente. Ele destaca a mesologia do local, ressaltando a dificuldade de definição de qual agente é preponderante nesse ambiente. Euclides da Cunha descreve a região como selvagem, com uma flora extravagante e um clima instável, marcado por intermitências de cheias e estiagens. Essas condições climáticas extremas influenciam a vida dos habitantes do sertão nordestino, levando a um estilo de vida difícil e adaptado às adversidades do ambiente.'

In [35]:
parent_chain_retriever.invoke('Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?')

'Euclides da Cunha descreve o sertanejo como um indivíduo forte, resistente e adaptado às adversidades do ambiente em que vive. Ele destaca a aparência desgraciosa e desengonçada do sertanejo, mas ressalta sua força física e resistência. O sertanejo é descrito como alguém que se adapta às condições climáticas extremas, como as intermitências de cheias e estiagens, e que enfrenta desafios como a friagem repentina que ocorre na região. Euclides da Cunha relaciona as características da população sertaneja com o ambiente em que vivem, destacando a resistência física e a adaptabilidade do sertanejo às condições adversas do sertão.'

In [36]:
parent_chain_retriever.invoke('Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?')

'Segundo Euclides da Cunha, o contexto histórico e político que levou à Guerra de Canudos foi marcado por uma série de expedições militares enviadas para destruir a cidade de Belo Monte, em Canudos, que abrigava cerca de trinta mil habitantes. Essas expedições foram motivadas pela derrota da terceira expedição militar comandada pelo Coronel Antônio Moreira César, que servia aos projetos políticos do governo de Floriano Peixoto. A guerra de Canudos foi vista como uma reação monárquica para solapar as instituições republicanas, levando à preparação de quatro expedições punitivas contra a comunidade de Canudos. Euclides da Cunha acompanhou as ações militares da quarta expedição como jornalista correspondente do jornal O Estado de São Paulo, e sua experiência no campo de batalha do sertão baiano influenciou sua visão sobre o Brasil e a guerra de Canudos.'

In [37]:
parent_chain_retriever.invoke('Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?')

'Euclides da Cunha descreve Antônio Conselheiro como um líder carismático e misterioso, que conseguiu reunir seguidores em torno de sua figura messiânica. Ele era visto como um homem irascível, visionário e de excelente caráter, com capacidade de negociar mesmo sendo analfabeto. Seu pai, Vicente Mendes Maciel, era descrito como um homem de honradez proverbial e rígido, que educou Antônio Conselheiro de forma a isolá-lo da turbulência da família. Antônio Conselheiro era retratado como um adolescente tranquilo e tímido, avesso à troça e dedicado aos negócios do pai. Sua vida seguia um caminho corretíssimo e calmo, até que após cuidar de suas irmãs solteiras, ele próprio se casou e teve um desfecho nefasto. Em meio a essas descrições, fica latente a sugestão de que Antônio Conselheiro poderia se tornar um líder em condições mais favoráveis, o que de fato aconteceu com a formação da comunidade de Canudos e sua resistência às expedições punitivas enviadas para destruí-la.'

In [38]:
parent_chain_retriever.invoke('Quais são os principais aspectos da crítica social e política presentes em "Os Sertões"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?')

'Os principais aspectos da crítica social e política presentes em "Os Sertões" estão relacionados à denúncia da injustiça social e à reflexão sobre a realidade do Brasil da época, marcada por transformações históricas como o fim da Guerra do Paraguai, a criação do Partido Republicano e a Abolição da Escravidão. O autor, Euclides da Cunha, apresenta uma visão crítica da sociedade brasileira, evidenciando a marginalização do sertanejo e a desigualdade social entre o interior e o litoral. Ele não defende a vida sertaneja contra a vida citadina, mas aponta para a necessidade de progresso e civilização. A obra reflete a preocupação do autor com uma realidade marcada pela barbárie e pela falta de compreensão dos governantes em relação ao Brasil profundo, esquecido e negligenciado.'